In [1]:
import torch
import torch.nn as nn
from torchvision import models, datasets, transforms
from torch.utils.data import DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)


Device: cuda


In [2]:
DATA_ROOT = "../fer2013"
TEST_DIR = f"{DATA_ROOT}/test"

transform_test = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

test_dataset = datasets.ImageFolder(TEST_DIR, transform=transform_test)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

print(f"Test samples: {len(test_dataset)}")
print(f"Classes: {test_dataset.classes}")


Test samples: 7178
Classes: ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']


In [3]:
# Recreate same model architecture
mobilenet = models.mobilenet_v2(weights=None)
num_ftrs = mobilenet.classifier[1].in_features
mobilenet.classifier[1] = nn.Linear(num_ftrs, 7)

# Load your fine-tuned weights
mobilenet.load_state_dict(torch.load("../trained_models/mobilenetv2_finetuned.pth", map_location=device))

mobilenet = mobilenet.to(device)
mobilenet.eval()

print("✅ Fine-tuned MobileNetV2 loaded successfully!")


✅ Fine-tuned MobileNetV2 loaded successfully!


In [4]:
correct, total = 0, 0
mobilenet.eval()

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = mobilenet(images)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

test_accuracy = 100 * correct / total
print(f"✅ Test Accuracy of loaded MobileNetV2: {test_accuracy:.2f}%")


✅ Test Accuracy of loaded MobileNetV2: 67.08%
